In [1]:
import sys
import os
sys.path.append('/Users/masahiro/song_recommender/music_recommender/')

In [2]:
#import featurization as feat
import k_means
from testing import evaluate, evaluate_k_means_model, evaluate_fuzzy_model

In [3]:
import numpy as np 
import pandas as pd 
from math import floor
import json
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pickle
from sklearn.preprocessing import MinMaxScaler
import statistics as st
import skfuzzy as fuzz
import os
import pickle

In [4]:
root_path = os.getcwd()[:-5]
filepath = f"{root_path}/playlist_data/sampledata/"
filename = 'mpd.slice.0-999.json'
fpath_name = f"{filepath}{filename}"
with open(fpath_name) as data_file:    
    data = json.load(data_file)

playlists_df = pd.json_normalize(data['playlists'])
    
filename = 'song.slice.0-49999.csv'
fpath_name = f"{filepath}{filename}"

songs_df = pd.read_csv(fpath_name)
songs_df = songs_df.iloc[:,2:]

filename = 'data_with_genres.csv'
fpath_name = f"{filepath}{filename}"
new_df = pd.read_csv(fpath_name)

In [5]:
model_name = 'fuzzy_all_model.pkl'
model = pickle.load(open(model_name, 'rb'))

In [7]:
playlists = playlists_df
features = ['danceability', 'energy', 'key', 'loudness',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'playlist_description_encoded', 'sentiment']
#features = [feat for feat in features if feat != 'key']

In [12]:
for feat in features:
    model_name = 'model_'+feat+'.pkl'
    model = pickle.load(open(model_name, 'rb'))
    
    result = evaluate_k_means_model(model, [feat], playlists, n_suggestions=10, val_ratio=0.5)
    print(feat, result)

danceability {'artist_match_rate': 0.012822222222222198, 'track_match_rate': 0.002122222222222223}
energy {'artist_match_rate': 0.013944444444444416, 'track_match_rate': 0.002700000000000001}
key {'artist_match_rate': 0.010455555555555537, 'track_match_rate': 0.0009999999999999998}
loudness {'artist_match_rate': 0.00889999999999999, 'track_match_rate': 0.0012}
speechiness {'artist_match_rate': 0.011366666666666652, 'track_match_rate': 0.0014000000000000004}
acousticness {'artist_match_rate': 0.00984444444444443, 'track_match_rate': 0.002500000000000001}
instrumentalness {'artist_match_rate': 0.008699999999999989, 'track_match_rate': 0.0007999999999999999}
liveness {'artist_match_rate': 0.008211111111111101, 'track_match_rate': 0.0013}
valence {'artist_match_rate': 0.010222222222222204, 'track_match_rate': 0.0017000000000000001}
tempo {'artist_match_rate': 0.007316666666666658, 'track_match_rate': 0.0013}


In [23]:
evaluate_k_means_model(model, features, playlists, n_suggestions=10, val_ratio=0.5)

{'artist_match_rate': 0.008088888888888878,
 'track_match_rate': 0.0017000000000000003}

In [7]:
cntr, u = model.values()

In [8]:
evaluate_fuzzy_model(features, playlists, n_suggestions=10, num_clusters = 30, refinement=3, val_ratio=0.5)

finished 0
finished 10
finished 20
finished 30
finished 40
finished 50
finished 60
finished 70
finished 80
finished 90
finished 100
finished 110
finished 120
finished 130
finished 140
finished 150
finished 160
finished 170
finished 180
finished 190
finished 200
finished 210
finished 220
finished 230
finished 240
finished 250
finished 260
finished 270
finished 280
finished 290
finished 300
finished 310
finished 320
finished 330
finished 340
finished 350
finished 360
finished 370
finished 380
finished 390
finished 400
finished 410
finished 420
finished 430
finished 440
finished 450
finished 460
finished 470
finished 480
finished 490
finished 500
finished 510
finished 520
finished 530
finished 540
finished 550
finished 560
finished 570
finished 580
finished 590
finished 600
finished 610
finished 620
finished 630
finished 640
finished 650
finished 660
finished 670
finished 680
finished 690
finished 700
finished 710
finished 720
finished 730
finished 740
finished 750
finished 760
finished 7

{'artist_match_rate': 0.031642583470169704,
 'track_match_rate': 0.006241379310344828}

In [16]:
songs = songs_df.select_dtypes(include=np.number)

In [17]:
columns = songs.columns

[columns.get_loc(feat) for feat in features]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]

In [13]:
# Create partial playlists to test the model on

partial_playlist = playlists_df.tracks.apply(lambda x: x[:max(1,floor(len(x)*(1-0.5)))])
partial_playlist = partial_playlist.to_frame()
partial_playlist['pid'] = range(1000)
partial_playlist

,tracks,pid
0,"[{'pos': 0, 'artist_name': 'Missy Elliott', 't...",0
1,"[{'pos': 0, 'artist_name': 'Survivor', 'track_...",1
2,"[{'pos': 0, 'artist_name': 'Hoody', 'track_uri...",2
3,"[{'pos': 0, 'artist_name': 'Camille Saint-Saën...",3
4,"[{'pos': 0, 'artist_name': 'The Smashing Pumpk...",4
...,...,...
995,"[{'pos': 0, 'artist_name': 'Katrina', 'track_u...",995
996,"[{'pos': 0, 'artist_name': 'PARTYNEXTDOOR', 't...",996
997,"[{'pos': 0, 'artist_name': 'LoveRance', 'track...",997
998,"[{'pos': 0, 'artist_name': 'Hunter Hayes', 'tr...",998


In [15]:
n_suggestions = 10

In [8]:
# Evaluation of K-means using song features only

# Normalize features
songs = songs_df.select_dtypes(include=np.number).to_numpy()
minmax = MinMaxScaler()
songs = minmax.fit_transform(songs)

# Fit the model
model = k_means.kmeans_fit(songs, N_clusters=50)



In [11]:
model.labels_[:50]

array([37, 37, 48, 37, 32, 32,  1, 27, 37, 37,  0, 37,  3,  3,  2,  3, 47,
       11, 14, 32, 14, 14, 14,  3, 46,  3, 24, 20,  3, 24, 35, 44, 25, 36,
       32, 34, 14, 33, 47, 27, 43, 14, 18, 34, 36, 36,  0, 36, 47, 27],
      dtype=int32)

In [11]:
def k_means_rec(playlists):
    
    recs = {'pid':[], 'tracks':[]}
    
    for i in range(len(playlists)):
        
        recs['pid'].append(playlists.pid[i])
                           
        playlist = playlists.tracks[i]
        uri_list = [track['track_uri'] for track in playlist]
        
        playlist = songs[[uri in uri_list for uri in song_df['uri']]]
        
        # Predict the cluster
        clusters = model.predict(playlist)
        cluster = st.mode(clusters)

        # Suggest songs from the cluster
        
        recs['tracks'].append(song_df[model.labels_ == cluster].sample(n_suggestions).to_dict(orient='records'))
        
    return pd.DataFrame(recs)

def k_means_rec_artist_priority(playlists):
    
    recs = {'pid':[], 'tracks':[]}
    
    for i in range(len(playlists)):
        
        recs['pid'].append(playlists.pid[i])
                           
        playlist = playlists.tracks[i]
        uri_list = [track['track_uri'] for track in playlist]
        
        playlist = songs[[uri in uri_list for uri in song_df['uri']]]
        playlist_df = song_df[[uri in uri_list for uri in song_df['uri']]]
        
        artists = playlist_df.artist_name.values

        
        # Predict the cluster
        clusters = model.predict(playlist)
        cluster = st.mode(clusters)

        # Suggest songs from the cluster
        
        crit1 = song_df[[artist in artists for artist in song_df['artist_name'].values]]
        crit2 = song_df[model.labels_ == cluster]
        
        crit = pd.merge(crit1, crit2, how='inner', on='track_uri', suffixes=('', '_y'))
        n = len(crit)
        
        recs['tracks'].append(crit.sample(min(n,n_suggestions)).to_dict(orient='records'))
        
    return pd.DataFrame(recs)


def baseline_rec(playlists):
    
    recs = {'pid':[], 'tracks':[]}
    
    for i in range(len(playlists)):
        
        recs['pid'].append(playlists.pid[i])
                           
        playlist = playlists.tracks[i]
        uri_list = [track['track_uri'] for track in playlist]
        
        playlist = songs_df[[uri in uri_list for uri in songs_df['uri']]]
        
        artists = playlist.artist_name.values

        # Suggest songs with the same artist
        
        recs['tracks'].append(songs_df[[artist in artists for artist in songs_df['artist_name'].values]].sample(n_suggestions, replace=True).to_dict(orient='records'))
        
    return pd.DataFrame(recs)

In [201]:
recs = k_means_rec_artist_priority(partial_playlist)

In [16]:
recs_b = baseline_rec(partial_playlist)

In [204]:
evaluate(recs)

{'artist_match_rate': 0.14291666666666664,
 'track_match_rate': 0.013166666666666667}

In [24]:
evaluate(recs_b, from_pid=True)

{'artist_match_rate': 0.11949206349206343,
 'track_match_rate': 0.01708333333333334}